In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 1 `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

### Our Final set of **occasions** for the category 1 bin is (after iterations): 
**`Lunch`**
	- 1 Meal at least
	- 1 Drink
		○ More common no alcohol
**`Dinner`**
- Time label
    - `Late`
- 1 Meal at least
- 1 Drink

**`Drinking`**
- Time label
    - `Afternoon`
    - `Dinner`
    - `Late`
- More drink than food
    - Ratio of 3+:1+
- But more common no food at all
- More meal steps (because of the drinks)
- Longer dwell time
- If less drinks, but hard liquer, or more drinks but beer

**`Casual Drink & meal`**
- Time label
    - `Afternoon`
    - `Dinner`
    - `Late`
- Usually starts with drink/beer
- More drinks than food
    - Ratio of about 2:1
- Food might not be ordered in first step
- Maybe even last step


### Something to think about:
- `Lunch` vs `dinner`
- `how long do singles stay at a table`
- `alcohol versus beer`
- `food versus drinks`

----------

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [3]:
hockey_path = '../data/hockey_3_text_processed.csv'
silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [4]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

In [115]:
sil = pd.read_csv(silvester_path)

Fix `datetime` features:

In [5]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [6]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

In [15]:
hockey.order_id.value_counts().keys()

Int64Index([521095549, 519477313, 514736044, 511415825, 520250633, 511378511,
            512851855, 516620908, 521100233, 525661404,
            ...
            520105238, 519527805, 514495361, 524862299, 514693984, 523514709,
            513545041, 512578876, 521098041, 511500325],
           dtype='int64', length=118664)

In [7]:
hockey = hockey[hockey.guest_count == 1]
valentine = valentine[valentine.guest_count == 1]
silvester = silvester[silvester.guest_count == 1]



In [13]:
def total_meal_steps(data):
    orders = {}
    order_ids = list(data.order_id.value_counts().keys())

    for order in order_ids:
        orders[order] = {}
            
        df = data[data.order_id == order]
        df = df.sort_values(by="order_item_time")
        last_meal_step = list(df.meal_step.sort_values())[-1]

        orders[order]['total_meal_steps'] = last_meal_step

        # from sit down to first order
        meal_flows = [(i * 4 + 1) for i in list(df.meal_flow_step)]

        orders[order]['sit_to_order'] = meal_flows[0]
        orders[order]['first_to_second_order'] = 0
        if len(meal_flows) > 1:
            orders[order]['first_to_second_order'] = meal_flows[1] - meal_flows[0]

        total_diff_flows = []
        prev_flow = 0
        for flow in meal_flows:
            if flow == prev_flow:
                continue
            total_diff_flows.append(flow - prev_flow)
            prev_flow = flow

        # avg time between orders
        avg_steps = sum(total_diff_flows)/len(total_diff_flows)
        orders[order]['avg_time_between_steps'] = avg_steps

    return orders

order_feats = total_meal_steps(hockey)
hockey['total_meal_steps'] = hockey.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
hockey['first_to_second_order'] = hockey.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
hockey['avg_time_between_steps'] = hockey.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])

order_feats = total_meal_steps(valentine)
valentine['total_meal_steps'] = valentine.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
valentine['first_to_second_order'] = valentine.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
valentine['avg_time_between_steps'] = valentine.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])

order_feats = total_meal_steps(silvester)
silvester['total_meal_steps'] = silvester.order_id.apply(lambda x:order_feats[x]['total_meal_steps'])
silvester['first_to_second_order'] = silvester.order_id.apply(lambda x:order_feats[x]['first_to_second_order'])
silvester['avg_time_between_steps'] = silvester.order_id.apply(lambda x:order_feats[x]['avg_time_between_steps'])


In [37]:
len(order_feats)

1

Lets only look at interesting features

In [9]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step','total_meal_steps',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week','max_items_per_step',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [10]:
hockey = hockey[features]
silvester = silvester[features]
valentine = valentine[features]

In [222]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,3,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [11]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [12]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

In [18]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  49664  tables in the Hockey data


In [19]:
print("We have ", len(valentine_tables), " tables in the Valentine data")

We have  15485  tables in the Valentine data


In [20]:
print("We have ", len(silvester_tables), " tables in the Silvester data")

We have  35027  tables in the Silvester data


In [13]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [13]:
COUNT = 10

In [12]:
import random

In [15]:
def get_tables_gc_1(tables):
    return tables[tables['guest_count'] == 1]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [16]:
hockey_1gc_tables = get_tables_gc_1(hockey_tables)
print("We have {} tables of a total of 1 guests".format(len(hockey_1gc_tables)))

We have 48146 tables of a total of 1 guests


In [58]:
#picked_hockey_tables = random.sample(list(hockey_1gc_tables.index), COUNT)

In [55]:
picked_hockey_tables = [512690383,
 521702519,
 521093892,
 520105316,
 521783372,
 521769692,
 512852707,
 525538989,
 514327163,
 520255453,
 511390190,
 525643037,
 525621948,
 525122551,
 521589076,
 514577448,
 512769146]

## Creating TEST Set

In [26]:
picked_hockey_tables_test = random.sample(list(hockey_tables.index), 10)
picked_hockey_tables_test

[520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

In [34]:
test_h = [520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

# DEV Annotations

# Table 1

In [26]:
#512690383
hockey[hockey.index == 512690383].sort_values(by=['order_item_time'])

,bar_id,order_time,order_time_closed,order_item_time,order_item_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,total_orders_category_id_1.0,total_orders_category_id_2.0,total_orders_category_id_3.0,total_orders_category_id_4.0,total_orders_category_id_5.0,total_orders_category_id_6.0,total_orders,order_day_of_week,is_weekend,sharable,kids_meal,birthday,order_hour,period_of_day,order_minute,order_item_minute,order_item_hour,order_close_minute,order_close_hour,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,total_sales_before_tax,total_sales_inc_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:10:00,51269038300,1.0,5.95,6.724694,1,QUESA (VEG),2.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,0,0,0,17,afternoon,10,10,17,23,18,1.225,1,7,17,0,20.43,23.09
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:16:00,51269038301,1.0,11.50,12.997308,1,FORBES MARG DBL,3.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,0,0,0,17,afternoon,10,16,17,23,18,1.225,2,7,17,1,20.43,23.09
512690383,17650,2019-04-11 17:10:00,2019-04-11 18:23:30,2019-04-11 17:52:00,51269038302,0.5,2.98,3.367998,1,MINI DONUTS,2.0,0,0,0.0,0,2,1.0,0,0,0,3,Thursday,False,1,0,0,17,afternoon,10,52,17,23,18,1.225,3,7,17,10,20.43,23.09


**April 11th (Thursday), 17:10 - 18:23, a total of 1.2 hours, 3 meal steps**

**Meal Steps Description:**
1. Start
  - Meal (not very heavy or big)
      - Quesa
2. Still Start (6 minutes after)
  - Drink
      - Margarhita
3. Dessert, last order (~40+ minutes later)
  - Dessert
      - mini donutes
      
**Total drinks**: 1 cocktail, **total meals**: 1 main dish, 1 dessert

**`after work meal`** / **Casual drink and Meal**

`afternoon`, `small_meal`, `drink`

**Explanation**: Because of the time and the size of the meal, to me it seems like what a person after work would go and eat. Maybe even go to dinner after, but this seems like a "light" meal after work.

**Features that helped**: `type of meal` ("quesadia" is not really a main course), `price of meal` (5-6 $), and `time of meal` (5pm). 

# Table 2

In [163]:
hockey[hockey.index == 521702519].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_serving_type_id,beer_volume,total_orders,order_day_of_week,is_weekend,order_hour,period_of_day,order_item_minute,sharable,kids_meal,birthday,total_sales_before_tax,beer_brand_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,total_sales_inc_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:28:00,HENDRICKS,3.0,0.788333,1,7,11,0,0,0.0,2,Thursday,False,15,afternoon,28,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91
521702519,2019-04-18 15:28:00,2019-04-18 16:15:18,2019-04-18 15:47:00,HENDRICKS,3.0,0.788333,2,7,11,4,0,0.0,2,Thursday,False,15,afternoon,47,0,0,0,18.5,0,1.0,9.25,10.455,1,20.91


**April 18th (Thursday), 15:28 - 16:15, total 0.7 hours, 2 meal steps**

**Meal Steps Description:**
1. Start
   - Drink
       - Gin
2. 20 Minutes Later:
   - Drink
       - Gin
       
**Total drinks**: 2 liquer drinks

**`Afternoon Drinking`**

`hard_liquer`, `afternoon`, `dwell_time`

**Explanation**: This is actually a very consistent person, he ordered two drinks exactly 20 minutes apart, and the table was closed ~20 minutes after the second drink.

**Features**: `time of meal` (3pm), `type of drink` (category 3, gin - hard liquer), `dwell time` (stayed ~40 min).

# Table 3

In [182]:
#521093892
hockey[hockey.index == 521093892].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 22:42:29,MOSCOW MULE DBL,3.0,2.356944,1,0,33,9,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-17 23:17:10,MOSCOW MULE DBL,3.0,2.356944,2,0,33,17,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,15.75
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:04:36,VODKA RED BULL,3.0,2.356944,3,0,33,28,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,9.50
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,INCEPTION BOMB,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,0.28,4.06
521093892,2019-04-17 22:04:00,2019-04-18 00:25:25,2019-04-18 00:25:25,FIREBALL,3.0,2.356944,4,0,33,33,0.0,Wednesday,False,1,hang_out,0,0,0,51.01,1.00,5.95


**April 17th (Wednesday), 22:04 - 00:25, total of 2.35 hours, 4 meal steps.**

**Meal Steps Description:**
1. Start (40 minutes after sit down)
    - Drink
        - Moscow Mule (cocktail type drink)
2. 35 minutes after
    - Drink
        - Moscow Mule
3. 45 minutes after (1.15 hrs since person started drinking). (Notice the change to harsher liquer)
    - Drink
        - Vodka redbull (less of a sit down bar, more of a "dance" bar, or "Getting drunk bar", as opposed to previous drinks)
4. 20 minutes after that (~2 hours since started drinking)
    - Drink
        - Inception Bomb (another weird whisky shot)
        - Fireball (sweet whisky shot)
     (two for the road?)

**Total drinks: 2 cocktails, 1 mixed drink, 2 shots**

**`drinking`**

`late_night`, `hard_liquer`, `quantity of drinks`, `dwell_time`

**Explanation**: Its hard to say if this person was watching a game or something, or was ordering at a bar and sitting with a group. I would guess that the person is by themselves, since its a bit weird to keep ordering single drinks for yourself the whole night, even shots? Although at the end the person orders two whisky shots... hmm. Still I would say a single person drinking at night, could be more of a dance bar because of the type of drinks.
(its interesting also to note that the person started with classy drinks, and ended with whisky :P)

**Features**: `time of the order` and `closing time` (started at 10pm and ended past midnight), `quantity of drinks` (had a total of 5 drinks)

# Table 4

In [52]:
hockey[hockey.index == 520105316].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:17:55,water.,4.0,0.668889,1,2,7,8,0,0.000,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,0.00
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:17:55,water.,4.0,0.668889,1,2,7,8,0,0.000,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,0.00
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:24:42,goose.ipa.xl,1.0,0.668889,2,2,7,8,1,0.591,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,10.30
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:24:42,goose.ipa.xl,1.0,0.668889,2,2,7,8,1,0.591,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,10.30
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:26:33,faj.two.steak,2.0,0.668889,2,2,7,8,1,0.000,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,36.97
520105316,2019-04-16 17:17:00,2019-04-16 17:57:08,2019-04-16 17:26:33,faj.for.two,2.0,0.668889,2,2,7,8,1,0.000,Tuesday,4,False,1,afternoon,0,0,0,57.57,1.0,0.00


** Mistake in the category **

We can add a rule for this

Notice there are 2 waters, and four items ordered in the same meal step (2). Two drinks and two meals (or a single large meal).


# Table 5

In [186]:
hockey[hockey.index == 521783372].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,SMOKIN.SALMON,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,23.96
521783372,2019-04-18 21:01:00,2019-04-18 21:20:02,2019-04-18 21:01:14,Salmon.Rice/Veg,2.0,0.317222,1,4,4,0,0.0,Thursday,False,1,hang_out,0,0,0,23.96,1.0,0.00


**April 18th (Thursday), 21:01 - 21:20, total of 0.3 hours, and 1 meal steps.**

**Meal Steps Description:**
1. Start 
    - Dinner (classic)
        - Salmon
        
**Total meals: 1 large/dinner meal**

**`dinner`**

`dwell_time`, `food_category_only`, `dinner_time`.

**Explanation**: Not much to explain, very classic single person dinner.


# Table 6

In [187]:
hockey[hockey.index == 521769692].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 18:41:00,LRG BONESHAKER,1.0,2.056667,1,19,29,0,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:24:00,BUF CHIC SALAD,2.0,2.056667,2,19,29,10,0.000,Thursday,False,1,dinner,0,0,0,29.5,1.0,15.00
521769692,2019-04-18 18:41:00,2019-04-18 20:44:24,2019-04-18 19:25:00,LRG BONESHAKER,1.0,2.056667,2,19,29,10,0.591,Thursday,False,1,dinner,0,0,0,29.5,1.0,7.25


**April 18th (Thursday), 18:41 - 20:44, total of 2 hours, 2 meal steps.**

**Meal Steps Description:**

1. Start:
   - Drink
       - Half a liter of beer (ordered when sat down)
2. ~40 minutes after
   - Drink
       - Half a liter of beer (ordered when sat down)
   - Meal
       - Salad

**total drinks**: 2 half liter beers, total meals: 1 salad

**`casual drink & meal`** - (I guess similar to relaxing alone ?)

`dwell_time`, `beer`, `time_between_drinks`, `dinner_time`

**Explanation**: I didn't want to say casual dinner, because the person first ordered a beer, and drank the beer for 40 minutes. I would say that he was watching a game or something, and after 40 minutes (maybe half time?) ordered food and another drink. Especially since he then goes on to sit another 1.2 hours with a beer and salad.. I mean no one takes that long to eat a salad unless they're donig someting else.

**Features**:  The fact that there was a long period of time for two beers. The time it took between beers, the time of the table, dinner time, and the light meal.



# Table 7

In [188]:
hockey[hockey.index == 512852707].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-11 23:40:03,GUINNESS.Pt,1.0,1.353889,1,0,20,0,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 00:41:03,GUINNESS.Pt,1.0,1.353889,2,0,20,15,0.562,Thursday,False,1,night,0,0,0,27.57,1.0,8.82
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,$Sub.Swt.Pot.Fry-,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,0,0,0,27.57,0.5,1.49
512852707,2019-04-11 23:40:00,2019-04-12 01:01:14,2019-04-12 01:01:14,FINGERS.SAUCED,2.0,1.353889,3,0,20,20,0.000,Thursday,False,1,night,1,0,0,27.57,0.5,8.44


**April 11th (Thursday), 23:40 - 01:01, total of 1.35 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Guinnes, half a liter
2. 1hr later
    - drink
        - Guiness, half a liter
3. 20 minutes later
    - meal
        - chicken fingers, and sweet potato fries
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

`late casual drink & meal`

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Table 8

In [189]:
hockey[hockey.index == 525538989].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 17:44:35,KEITHS.XL,1.0,1.696389,1,9,23,0,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:27:36,KEITHS.XL,1.0,1.696389,2,9,23,10,0.591,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,10.01
525538989,2019-04-23 17:44:00,2019-04-23 19:25:47,2019-04-23 18:46:51,CHEESE.CHIKN.MAC,2.0,1.696389,3,9,23,14,0.000,Tuesday,False,1,afternoon,0,0,0,39.99,1.0,19.97


**April 23rd (Tuesday), 17:44 - 19:25, total of 1.7 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Keiths, half a liter
2. 45 min later
    - drink
        - Keiths, half a liter
3. 20 minutes later
    - meal
        - chicken mac and cheese (bar food.. amarican bar food. I miss mac&cheese)
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

**`casual drink & meal`**

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

### Table 9

In [63]:
hockey[hockey.index == 514327163].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
514327163,2019-04-13 14:01:00,2019-04-13 14:58:43,2019-04-13 14:01:05,trad.guac,2.0,0.961944,1,2,9,13,0,0.0,Saturday,2,True,1,afternoon,1,0,0,28.43,1.0,8.99
514327163,2019-04-13 14:01:00,2019-04-13 14:58:43,2019-04-13 14:02:03,cherry.coke,4.0,0.961944,1,2,9,13,0,0.0,Saturday,2,True,1,afternoon,0,0,0,28.43,1.0,4.45
514327163,2019-04-13 14:01:00,2019-04-13 14:58:43,2019-04-13 14:22:02,dbl.plain.burg,2.0,0.961944,2,2,9,13,4,0.0,Saturday,2,True,1,afternoon,0,0,0,28.43,1.0,14.99


**->LUNCH**

### Table 10

In [64]:
hockey[hockey.index == 520255453].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
520255453,2019-04-16 22:47:00,2019-04-17 00:07:24,2019-04-16 22:47:56,l.short.nacho.veg,2.0,1.34,1,1,19,19,0,0.0,Tuesday,1,False,1,hang_out,0,0,0,5.48,1.0,5.48


**->MUNCH**

### Table 11

In [65]:
hockey[hockey.index == 511390190].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
511390190,2019-04-10 19:46:00,2019-04-10 20:15:49,2019-04-10 19:46:12,veggie burger,2.0,0.496944,1,1,7,7,0,0.0,Wednesday,1,False,1,dinner,0,0,0,5.95,1.0,5.95


**->DINNER**

### Table 12

In [66]:
hockey[hockey.index == 525643037].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
525643037,2019-04-23 21:54:00,2019-04-23 23:34:40,2019-04-23 22:38:12,w.ej.blush.gl,6.0,1.677778,1,2,0,25,11,0.0,Tuesday,2,False,1,hang_out,0,0,0,19.03,1.0,6.54
525643037,2019-04-23 21:54:00,2019-04-23 23:34:40,2019-04-23 23:34:40,olmeca.teq,3.0,1.677778,2,2,0,25,25,0.0,Tuesday,2,False,1,hang_out,0,0,0,19.03,1.0,5.00
525643037,2019-04-23 21:54:00,2019-04-23 23:34:40,2019-04-23 23:34:40,quesadillas,2.0,1.677778,2,2,0,25,25,0.0,Tuesday,2,False,1,hang_out,1,0,0,19.03,0.5,7.49


**->CASUAL_DRINK_AND_MEAL**

### Table 13

In [67]:
hockey[hockey.index == 525621948].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
525621948,2019-04-23 22:10:00,2019-04-23 22:52:49,2019-04-23 22:10:14,el tues pbr,1.0,0.713611,1,1,10,10,0,0.473,Tuesday,2,False,1,hang_out,0,0,0,13.2,1.0,5.0
525621948,2019-04-23 22:10:00,2019-04-23 22:52:49,2019-04-23 22:11:03,nachos for,2.0,0.713611,1,1,10,10,0,0.000,Tuesday,2,False,1,hang_out,1,0,0,13.2,1.0,8.2


**->CASUAL_DRINK_MEAL**

### Table 14

In [76]:
hockey[hockey.index == 525122551].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
525122551,2019-04-22 22:54:00,2019-04-22 23:31:41,2019-04-22 22:54:31,faj.for.two,2.0,0.628056,1,1,9,9,0,0.0,Monday,2,False,1,hang_out,0,0,0,36.97,1.0,0.00
525122551,2019-04-22 22:54:00,2019-04-22 23:31:41,2019-04-22 22:54:31,faj.two.steak,2.0,0.628056,1,1,9,9,0,0.0,Monday,2,False,1,hang_out,0,0,0,36.97,1.0,36.97


**->DINNER**

### Table 15

In [69]:
hockey[hockey.index == 521589076].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
521589076,2019-04-18 12:46:00,2019-04-18 13:33:29,2019-04-18 12:46:59,pop.,4.0,0.791389,1,2,10,11,0,0.0,Thursday,1,False,1,lunch,0,0,0,13.74,1.0,3.99
521589076,2019-04-18 12:46:00,2019-04-18 13:33:29,2019-04-18 12:50:59,ln.veg.quesa.,2.0,0.791389,2,2,10,11,1,0.0,Thursday,1,False,1,lunch,0,0,0,13.74,1.0,9.75


**->LUNCH**

### Table 16

In [70]:
hockey[hockey.index == 514577448].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
514577448,2019-04-13 20:39:58,2019-04-13 21:34:19,2019-04-13 20:40:07,water,4.0,0.905833,1,1,13,13,0,0.0,Saturday,2,True,1,dinner,0,0,0,11.19,1.0,0.00
514577448,2019-04-13 20:39:58,2019-04-13 21:34:19,2019-04-13 20:40:07,meatloaf,2.0,0.905833,1,1,13,13,0,0.0,Saturday,2,True,1,dinner,0,0,0,11.19,1.0,11.19


**->DINNER**

### Table 17

In [79]:
hockey[hockey.index == 512769146].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
512769146,2019-04-11 20:39:11,2019-04-11 21:30:12,2019-04-11 20:39:30,water,4.0,0.850278,1,2,0,11,0,0.0,Thursday,2,False,1,dinner,0,0,0,22.98,1.0,0.00
512769146,2019-04-11 20:39:11,2019-04-11 21:30:12,2019-04-11 20:39:30,great reuben,5.0,0.850278,1,2,0,11,0,0.0,Thursday,2,False,1,dinner,0,0,0,22.98,1.0,14.99
512769146,2019-04-11 20:39:11,2019-04-11 21:30:12,2019-04-11 21:26:18,chzcake plain,2.0,0.850278,2,2,0,11,11,0.0,Thursday,2,False,1,dinner,1,0,0,22.98,1.0,7.99


**->DINNER**

# Hockey TEST annotations

In [35]:
test_h

[520256354,
 514474566,
 522554624,
 522777463,
 519619252,
 519598706,
 512856854,
 520171895,
 520960843,
 520336377]

### Table 1

In [29]:
hockey[hockey.index == 520256354].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520256354,2019-04-16 19:46:00,2019-04-16 23:44:17,2019-04-16 19:46:00,POLAR ICE,3.0,3.971389,1,45,59,0,0.0,Tuesday,False,1,dinner,0,0,0,10.5,1.0,5.25
520256354,2019-04-16 19:46:00,2019-04-16 23:44:17,2019-04-16 20:42:00,POLAR ICE,3.0,3.971389,2,45,59,14,0.0,Tuesday,False,1,dinner,0,0,0,10.5,1.0,5.25


**->DRINKING**

### Table 2

In [36]:
hockey[hockey.index == 514474566].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514474566,2019-04-13 15:42:00,2019-04-13 16:44:42,2019-04-13 15:42:51,PINAPL.MARG,3.0,1.045,1,12,14,0,0.0,Saturday,True,1,afternoon,0,0,0,32.96,1.0,14.99
514474566,2019-04-13 15:42:00,2019-04-13 16:44:42,2019-04-13 15:54:45,HOLY.SMOKE.BURGR,2.0,1.045,2,12,14,2,0.0,Saturday,True,1,afternoon,0,0,0,32.96,1.0,17.97


**->Casual drink and meal**

### Table 3

In [38]:
hockey[hockey.index == 522554624].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,WATER.,4.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,0.00
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,BLURASP LEMONADE,3.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.67,12.37
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,WATER.,4.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,0.00
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:01:05,ORNG.DREAMSCL,3.0,1.191944,1,16,17,0,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,8.97
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:07:25,POUT.CLASSIC,2.0,1.191944,2,16,17,1,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,9.53
522554624,2019-04-19 16:01:00,2019-04-19 17:12:31,2019-04-19 16:07:25,FALAFEL-WICH,2.0,1.191944,2,16,17,1,0.0,Friday,True,1,afternoon,0,0,0,46.86,1.00,15.99


**->Not Category 1**

### Table 4

In [40]:
hockey[hockey.index == 522777463].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 00:36:28,HEINEKEN,1.0,0.402222,1,0,5,0,0.33,Saturday,True,1,night,0,0,0,17.75,1.0,7.25
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 00:42:24,CROWN ROYAL,3.0,0.402222,2,0,5,1,0.00,Saturday,True,1,night,0,0,0,17.75,1.0,7.25
522777463,2019-04-20 00:36:00,2019-04-20 01:00:08,2019-04-20 01:00:08,RED BULL STAFF,4.0,0.402222,3,0,5,5,0.00,Saturday,True,1,night,0,0,0,17.75,1.0,3.25


**->CASUAL_DRINK**

### Table 5

In [42]:
hockey[hockey.index == 519619252].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:06:17,BLOODY CAESAR,3.0,3.1175,1,21,46,0,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,9.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:11:10,ONION RING,2.0,3.1175,2,21,46,1,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,3.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 19:48:35,PNT BUDWEISER,1.0,3.1175,3,21,46,10,0.473,Monday,False,1,dinner,0,0,0,27.0,1.0,5.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 20:48:53,Bite,5.0,3.1175,4,21,46,25,0.000,Monday,False,1,dinner,0,0,0,27.0,1.0,3.0
519619252,2019-04-15 19:06:00,2019-04-15 22:13:03,2019-04-15 20:48:53,BEER N BITE,1.0,3.1175,4,21,46,25,0.473,Monday,False,1,dinner,0,0,0,27.0,1.0,7.0


**->Casual drink and meal** - this one was hard, I would say its casual becase of the time between drinks

### Table 6

In [43]:
hockey[hockey.index == 519598706].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
519598706,2019-04-15 22:33:00,2019-04-15 23:09:00,2019-04-15 22:36:50,CHK.PARMIGIANA,2.0,0.6,1,8,8,0,0.0,Monday,False,1,hang_out,0,0,0,20.87,1.0,20.87


**->Dinner**

### Table 7

In [44]:
hockey[hockey.index == 512856854].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:09:00,JUG CANADIAN,1.0,9.3475,1,119,139,0,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:09:00,JUG CANADIAN,1.0,9.3475,1,119,139,0,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 13:31:00,SHRIMP FAJITAS,2.0,9.3475,2,119,139,5,0.0000,Thursday,False,1,lunch,0,0,0,33.75,1.00,18.00
512856854,2019-04-11 13:09:00,2019-04-11 22:29:51,2019-04-11 14:31:00,JUG MILLSTREET,1.0,9.3475,3,119,139,20,0.4435,Thursday,False,1,lunch,0,0,0,33.75,0.25,5.25


**->Drinking** - weird one, who orders two half liters right off the bat?m

### Table 8

In [46]:
hockey[hockey.index == 520171895].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520171895,2019-04-16 20:23:00,2019-04-16 21:29:43,2019-04-16 20:23:46,QUESADILLAS,2.0,1.111944,1,16,16,0,0.0,Tuesday,False,1,dinner,1,0,0,7.48,1.0,7.48


**->Dinner**

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [81]:
valentine_1gc = get_tables_gc_1(valentine_tables)
print("We have {} tables of a total of 1 guests".format(len(valentine_1gc)))

We have 14921 tables of a total of 1 guests


In [84]:
picked_valentine_tables = random.sample(list(valentine_1gc.index), COUNT)

In [86]:
picked_valentine_tables = [434780854,
 434753224,
 447023765,
 434766693,
 446799837,
 448086739,
 447551853,
 447019925,
 447168624,
 447444332,
 435140033,
 448121768,
 434399752,
 446906440,
 447033893,
 447052748,
 447729251]

In [137]:
#TESTING
picked_valentine_test_tables = random.sample(list(valentine_1gc.index), COUNT)
picked_valentine_test_tables

[434752175,
 447067242,
 435110243,
 434681977,
 447595398,
 447608841,
 447650659,
 447372783,
 434735387,
 447074681]

In [138]:
test_v = [434752175,
 447067242,
 435110243,
 434681977,
 447595398,
 447608841,
 447650659,
 447372783,
 434735387,
 447074681]

# VALENTINE DEV ANNOTATIONS

## Table 1

In [206]:
valentine[valentine.index == 434780854].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,BEEFEATER**,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,MARTINI,3.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.97
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:13:00,WATER.,4.0,3.084167,1,9,44,0,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD THAI,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,0.00
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,BEEFEATER**,3.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.5,3.40
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 14:28:00,PAD.THAI.CHICKN,2.0,3.084167,2,9,44,3,0.000,Sunday,True,1,afternoon,0,0,0,62.09,1.0,20.74
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:10:00,BAR.MANGER.Pt,1.0,3.084167,3,9,44,13,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.72
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 15:44:00,RICKARD.RED.Pt,1.0,3.084167,4,9,44,21,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43
434780854,2019-02-10 14:13:00,2019-02-10 17:18:03,2019-02-10 16:40:00,RICKARD.RED.Pt,1.0,3.084167,5,9,44,35,0.473,Sunday,True,1,afternoon,0,0,0,62.09,1.0,8.43


**Feb 10th (Sunday), 14:13 - 17:18, total of ~3 hour, 5 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - martini
        - Beefeater ??? -cheap, maybe shot?
        - water
2. 15 min later
    - meal
        - pad thai chicken
    - drink
        - beefeater ???
3. 40 min later
    - drink
        - "bad manager" beer, half liter
      
4. 35 min later
    - drink
        - red pt, half liter
5. 1 hour later
    - drink
        - red pt, hald liter
        

**total drinks**: 4 or 6, depending if we cound beefeater, **total meals**: 1 meal.

**`DRINKING`**

`dwell_time`, `amount of alcohol`, `time between drinks`

**Explanation**: This one was not easy. 3 whole hours at a bar. at first it looks like a mistake, but then you see that there are actual steps, where you start with a drink, eat a meal, and finish with two more drinks. Hard to say what type of occasion this is, but I would say a afternoon meal and drink (not casual?), since there is a meal consumed, and a good amount of drink and beer consumed.

- I wanted to say casual at first, but maybe because of the martini and amount of total alcohol its not casual?.. but its 2pm :/


# Table 2

In [207]:
valentine[valentine.index == 434753224].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS..,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,0.00
434753224,2019-02-10 21:56:00,2019-02-10 22:25:51,2019-02-10 21:56:00,WINGS.5PC,2.0,0.4975,1,7,7,0,0.0,Sunday,True,1,hang_out,1,0,0,10.47,1.0,10.47


**Feb 10th (Sunday), 21:56 - 22:25, total of 0.5 hours (30 minutes), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - chicken fingers
        
**total meals**: 1 late night sharable meal

**`MUNCH`** - a sharble item late at night

`sharable`, `late_night`, `1_step/1 item`


**Explanation**: Looks like a late munch. I would even say maybe its a take out, then again it could be a person stopped for wings and then left.



# Table 3

In [211]:
valentine[valentine.index == 447023765].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:08:00,POP.,4.0,0.985556,1,4,14,0,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,3.39
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:12:00,FINGERS.SAUCED,2.0,0.985556,2,4,14,1,0.0,Wednesday,False,1,lunch,1,0,0,23.07,1.0,14.34
447023765,2019-02-13 13:08:00,2019-02-13 14:07:08,2019-02-13 13:49:00,BROWNIE.COOKIE,2.0,0.985556,3,4,14,10,0.0,Wednesday,False,1,lunch,0,0,0,23.07,1.0,5.34


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - soda pop
2. 4 min later
    - meal
        - chicken fingers
3. 35 minutes after
    - dessert
        - brownie

**total drinks**: 1 non alc drink, **total meals**: 1 lunch meal, and 1 dessert.

**`lunch`**

`lunch_time`, `meal`, `dwell_time`

**Explanation**: classic lunch, even went for dessert. Lunches shouldn't usually dwell too long. no alcohol and a single meal.

**Features**: time of meal (1pm), no alcohol, and spent about an hour.

# Table 4

In [209]:
valentine[valentine.index == 434766693].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 00:50:00,PNT AMSTERDAM,1.0,1.294444,1,1,18,0,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86
434766693,2019-02-10 00:50:00,2019-02-10 02:07:40,2019-02-10 02:01:00,PNT MILL ST,1.0,1.294444,2,1,18,17,0.591,Sunday,True,1,night,0,0,0,13.72,1.0,6.86


**Feb 10th (Tuesday), 00:50 - 02:07, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - amsterdam beer, half liter
2. 1 hr and 10 min later
    - drink
        - mills, half liter

**total drinks**: 2 beer drinks

**`late casual drink`** I think theres a difference between drinking late at night beer, and hard liquer.

    `time_between_drinks`, `beer`, `late_night` 

**Explanation**: pretty straight forward, late at night, drinking beer and taking its time. drinking a beer over an hour and then a second one.

**Features**: an hour between beers, having a total of two beers (seems like a common number of beers), and late night

#### 5. Table 446744761

In [210]:
valentine[valentine.index == 446799837].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:37:00,R.JT.MERLOT.Gl,6.0,1.335278,1,17,19,0,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,6.00
446799837,2019-02-12 18:37:00,2019-02-12 19:57:07,2019-02-12 18:46:00,BACON.FILET*,2.0,1.335278,2,17,19,2,0.0,Tuesday,False,1,dinner,0,0,0,36.71,1.0,30.71


**Feb 12th (Tuesday), 18:37 - 19:57, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Wine
2. 10 min later
    - meal
        - Bacon Filet (huh?.. fancy)

**total drinks**: 1 drink, **total meals**: 1 dinner meal.

**`dinner`** (maybe even a celebratory dinner, or "pinuk" dinner? "treat yo self" type dinner.

`wine`, `expensive_meal`, `dinner_time`

**Explanation**: classic dinner table, although important to note that the wine and filet makes it a bit more fancy, special occasion type:)

**Features**: expensive meal -(30$), dinner time (6pm), `wine`.

### Table 6

In [88]:
valentine[valentine.index == 448086739].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
448086739,2019-02-15 20:29:00,2019-02-15 21:49:12,2019-02-15 20:36:00,holy.smoke.burgr,2.0,1.336667,1,2,8,18,1,0.000,Friday,2,True,1,dinner,0,0,0,36.63,1.0,17.97
448086739,2019-02-15 20:29:00,2019-02-15 21:49:12,2019-02-15 20:36:00,guinness.pt,1.0,1.336667,1,2,8,18,1,0.562,Friday,2,True,1,dinner,0,0,0,36.63,1.0,9.33
448086739,2019-02-15 20:29:00,2019-02-15 21:49:12,2019-02-15 21:14:00,guinness.pt,1.0,1.336667,2,2,8,18,10,0.562,Friday,2,True,1,dinner,0,0,0,36.63,1.0,9.33


**->DINNER**

### Table 7

In [90]:
valentine[valentine.index == 447551853].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447551853,2019-02-14 22:22:00,2019-02-14 23:01:41,2019-02-14 22:22:00,keiths tall boy,1.0,0.661389,1,1,9,9,0,0.473,Thursday,1,False,1,hang_out,0,0,0,5.99,1.0,5.99


**->CASUAL_DRINK**

### Table 8

In [91]:
valentine[valentine.index == 447019925].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447019925,2019-02-13 12:55:20,2019-02-13 13:59:42,2019-02-13 12:55:35,pad thai,2.0,1.072778,1,1,16,16,0,0.0,Wednesday,1,False,1,lunch,0,0,0,11.55,1.0,11.55


**->LUNCH**

### Table 9

In [92]:
valentine[valentine.index == 447168624].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447168624,2019-02-11 15:24:00,2019-02-11 18:26:59,2019-02-11 16:08:00,jug mill st,1.0,3.049722,1,2,28,45,11,1.774,Monday,1,False,1,afternoon,0,0,0,22.55,1.00,19.25
447168624,2019-02-11 15:24:00,2019-02-11 18:26:59,2019-02-11 16:34:00,hh nacho,2.0,3.049722,2,2,28,45,17,0.000,Monday,1,False,1,afternoon,0,0,0,22.55,0.33,3.30


**->CASUAL_DRINK**

### Table 10

In [93]:
valentine[valentine.index == 447444332].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447444332,2019-02-14 16:29:53,2019-02-14 18:06:16,2019-02-14 16:30:27,oz steak frites,2.0,1.606389,1,1,23,23,0,0.0,Thursday,2,False,1,afternoon,0,0,0,38.98,1.0,24.99
447444332,2019-02-14 16:29:53,2019-02-14 18:06:16,2019-02-14 16:30:27,crispy calamari,2.0,1.606389,1,1,23,23,0,0.0,Thursday,2,False,1,afternoon,1,0,0,38.98,1.0,13.99


**->UNKNOWN** - only food items between lunch and dinner

### Table 11

In [94]:
valentine[valentine.index == 435140033].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
435140033,2019-02-10 15:30:00,2019-02-10 19:30:25,2019-02-10 15:30:00,bar.manger.mg,1.0,4.006944,1,3,21,59,0,0.355,Sunday,3,True,1,afternoon,0,0,0,42.69,1.0,7.06
435140033,2019-02-10 15:30:00,2019-02-10 19:30:25,2019-02-10 15:30:00,truffle.fries,2.0,4.006944,1,3,21,59,0,0.000,Sunday,3,True,1,afternoon,1,0,0,42.69,1.0,0.00
435140033,2019-02-10 15:30:00,2019-02-10 19:30:25,2019-02-10 15:30:00,sirl.frites,2.0,4.006944,1,3,21,59,0,0.000,Sunday,3,True,1,afternoon,0,0,0,42.69,1.0,21.97
435140033,2019-02-10 15:30:00,2019-02-10 19:30:25,2019-02-10 16:53:00,rickard.red.mg,1.0,4.006944,2,3,21,59,20,0.355,Sunday,3,True,1,afternoon,0,0,0,42.69,1.0,6.83
435140033,2019-02-10 15:30:00,2019-02-10 19:30:25,2019-02-10 18:06:00,keiths.mg,1.0,4.006944,3,3,21,59,38,0.355,Sunday,3,True,1,afternoon,0,0,0,42.69,1.0,6.83


**->CASUAL_DRINK**

### Table 12

In [96]:
valentine[valentine.index == 448121768].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
448121768,2019-02-15 21:59:00,2019-02-15 23:01:20,2019-02-15 22:11:00,ny.classic,2.0,1.038889,1,2,2,15,3,0.0,Friday,3,True,1,hang_out,0,0,0,35.09,1.0,31.10
448121768,2019-02-15 21:59:00,2019-02-15 23:01:20,2019-02-15 22:11:00,mashed,2.0,1.038889,1,2,2,15,3,0.0,Friday,3,True,1,hang_out,0,0,0,35.09,1.0,0.00
448121768,2019-02-15 21:59:00,2019-02-15 23:01:20,2019-02-15 22:11:00,jacks.veg,2.0,1.038889,1,2,2,15,3,0.0,Friday,3,True,1,hang_out,0,0,0,35.09,1.0,0.00
448121768,2019-02-15 21:59:00,2019-02-15 23:01:20,2019-02-15 22:53:00,pop.,4.0,1.038889,2,2,2,15,13,0.0,Friday,3,True,1,hang_out,0,0,0,35.09,1.0,3.99


**->DINNER**

### Table 13

In [97]:
valentine[valentine.index == 434399752].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
434399752,2019-02-10 00:30:00,2019-02-10 01:02:43,2019-02-10 00:30:36,bud light tap,1.0,0.545278,1,1,8,8,0,0.414,Sunday,1,True,1,night,0,0,0,6.25,1.0,6.25


**->CASUAL_DRINK**

### Table 14

In [99]:
valentine[valentine.index == 446906440].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
446906440,2019-02-12 23:28:21,2019-02-13 01:39:35,2019-02-12 23:28:21,wed tequila,3.0,2.187222,1,1,32,32,0,0.000,Tuesday,4,False,1,night,0,0,0,34.78,1.0,5.00
446906440,2019-02-12 23:28:21,2019-02-13 01:39:35,2019-02-12 23:28:21,corona,1.0,2.187222,1,1,32,32,0,0.710,Tuesday,4,False,1,night,0,0,0,34.78,2.0,11.94
446906440,2019-02-12 23:28:21,2019-02-13 01:39:35,2019-02-12 23:28:21,corona,1.0,2.187222,1,1,32,32,0,0.355,Tuesday,4,False,1,night,0,0,0,34.78,1.0,5.97
446906440,2019-02-12 23:28:21,2019-02-13 01:39:35,2019-02-12 23:28:21,welly gin,3.0,2.187222,1,1,32,32,0,0.000,Tuesday,4,False,1,night,0,0,0,34.78,1.0,11.87


**->NOT_1**

### Table 15

In [100]:
valentine[valentine.index == 447033893].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447033893,2019-02-13 14:39:00,2019-02-13 15:39:51,2019-02-13 14:39:00,sparkling water,4.0,1.014167,1,2,12,14,0,0.0,Wednesday,2,False,1,afternoon,0,0,0,30.9,1.0,7.50
447033893,2019-02-13 14:39:00,2019-02-13 15:39:51,2019-02-13 14:49:00,side.caesar,2.0,1.014167,2,2,12,14,2,0.0,Wednesday,2,False,1,afternoon,0,0,0,30.9,1.0,4.65
447033893,2019-02-13 14:39:00,2019-02-13 15:39:51,2019-02-13 14:49:00,bac.chs.burgr,2.0,1.014167,2,2,12,14,2,0.0,Wednesday,2,False,1,afternoon,0,0,0,30.9,1.0,18.75


**->UNKNOWN**

### Table 16

In [101]:
valentine[valentine.index == 447052748].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447052748,2019-02-13 16:36:32,2019-02-13 17:18:44,2019-02-13 16:36:44,fount pop,4.0,0.703333,1,1,10,10,0,0.0,Wednesday,2,False,1,afternoon,0,0,0,14.98,1.0,2.99
447052748,2019-02-13 16:36:32,2019-02-13 17:18:44,2019-02-13 16:36:44,ed fish chips,2.0,0.703333,1,1,10,10,0,0.0,Wednesday,2,False,1,afternoon,0,0,0,14.98,1.0,11.99


**->UNKNOWN**

### Table 17

In [102]:
valentine[valentine.index == 447729251].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447729251,2019-02-14 11:50:58,2019-02-14 12:13:02,2019-02-14 11:52:21,vegetarian burger,2.0,0.367778,1,1,5,5,0,0.0,Thursday,3,False,1,lunch,0,0,0,29.0,1.0,18.0
447729251,2019-02-14 11:50:58,2019-02-14 12:13:02,2019-02-14 11:52:21,garlic bread,2.0,0.367778,1,1,5,5,0,0.0,Thursday,3,False,1,lunch,1,0,0,29.0,1.0,7.0
447729251,2019-02-14 11:50:58,2019-02-14 12:13:02,2019-02-14 11:52:21,room service charge,2.0,0.367778,1,1,5,5,0,0.0,Thursday,3,False,1,lunch,0,0,0,29.0,1.0,4.0


**->LUNCH**

# VALENTINE TEST ANNOTATIONS

### TABLE 1

In [139]:
valentine[valentine.index == 434752175].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
434752175,2019-02-10 21:26:00,2019-02-10 22:59:25,2019-02-10 21:30:00,smokin.salmon,2.0,1.556944,1,1,22,23,1,0.0,Sunday,2,True,1,hang_out,0,0,0,23.96,1.0,23.96
434752175,2019-02-10 21:26:00,2019-02-10 22:59:25,2019-02-10 21:30:00,salmon.rice/veg,2.0,1.556944,1,1,22,23,1,0.0,Sunday,2,True,1,hang_out,0,0,0,23.96,1.0,0.00


**->DINNER**

### TABLE 2

In [140]:
valentine[valentine.index == 447067242].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447067242,2019-02-13 17:42:00,2019-02-13 18:36:41,2019-02-13 17:42:00,polar.ice..,3.0,0.911389,1,3,9,12,0,0.0,Wednesday,2,False,1,afternoon,0,0,0,35.28,1.0,0.00
447067242,2019-02-13 17:42:00,2019-02-13 18:36:41,2019-02-13 17:42:00,bloody.caesar,3.0,0.911389,1,3,9,12,0,0.0,Wednesday,2,False,1,afternoon,0,0,0,35.28,1.0,8.97
447067242,2019-02-13 17:42:00,2019-02-13 18:36:41,2019-02-13 17:46:00,trad.guac,2.0,0.911389,2,3,9,12,1,0.0,Wednesday,2,False,1,afternoon,1,0,0,35.28,0.5,4.50
447067242,2019-02-13 17:42:00,2019-02-13 18:36:41,2019-02-13 17:57:00,faj.chicken,2.0,0.911389,3,3,9,12,3,0.0,Wednesday,2,False,1,afternoon,0,0,0,35.28,1.0,21.81
447067242,2019-02-13 17:42:00,2019-02-13 18:36:41,2019-02-13 17:57:00,faj.for.,2.0,0.911389,3,3,9,12,3,0.0,Wednesday,2,False,1,afternoon,0,0,0,35.28,1.0,0.00


**DINNER**

### TABLE 3

In [141]:
valentine[valentine.index == 435110243].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
435110243,2019-02-11 20:47:00,2019-02-11 21:58:32,2019-02-11 20:47:00,roast.chickn,2.0,1.192222,1,1,17,17,0,0.0,Monday,3,False,1,dinner,0,0,0,34.96,1.0,19.97
435110243,2019-02-11 20:47:00,2019-02-11 21:58:32,2019-02-11 20:47:00,pinapl.marg,3.0,1.192222,1,1,17,17,0,0.0,Monday,3,False,1,dinner,0,0,0,34.96,1.0,14.99
435110243,2019-02-11 20:47:00,2019-02-11 21:58:32,2019-02-11 20:47:00,chikn.rice/veg,2.0,1.192222,1,1,17,17,0,0.0,Monday,3,False,1,dinner,0,0,0,34.96,1.0,0.00


**DINNER**

### TABLE 4

In [142]:
valentine[valentine.index == 434681977].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
434681977,2019-02-10 16:44:00,2019-02-10 18:44:05,2019-02-10 16:44:00,trad.guac,2.0,2.001389,1,2,24,29,0,0.0,Sunday,1,True,1,afternoon,1,0,0,22.12,0.5,4.50
434681977,2019-02-10 16:44:00,2019-02-10 18:44:05,2019-02-10 17:06:00,veggie.burger,2.0,2.001389,2,2,24,29,5,0.0,Sunday,1,True,1,afternoon,0,0,0,22.12,1.0,17.62


**DINNER**

### TABLE 5

In [144]:
valentine[valentine.index == 447595398].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447595398,2019-02-14 22:30:00,2019-02-14 23:29:18,2019-02-14 22:35:00,mojito rasp dbl,3.0,0.988333,1,2,8,14,1,0.0,Thursday,1,False,1,hang_out,0,0,0,19.5,1.0,13.0
447595398,2019-02-14 22:30:00,2019-02-14 23:29:18,2019-02-14 22:57:00,cuervo,3.0,0.988333,2,2,8,14,6,0.0,Thursday,1,False,1,hang_out,0,0,0,19.5,1.0,6.5


**DRINKING**

### TABLE 6

In [145]:
valentine[valentine.index == 447608841].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447608841,2019-02-14 16:28:00,2019-02-14 19:17:24,2019-02-14 16:36:00,r.sterl.vn.cs.gl,6.0,2.823333,1,3,8,41,2,0.0,Thursday,1,False,1,afternoon,0,0,0,34.06,1.0,9.79
447608841,2019-02-14 16:28:00,2019-02-14 19:17:24,2019-02-14 16:55:00,dbl.plain.burg,2.0,2.823333,2,3,8,41,6,0.0,Thursday,1,False,1,afternoon,0,0,0,34.06,1.0,15.29
447608841,2019-02-14 16:28:00,2019-02-14 19:17:24,2019-02-14 18:45:00,pecan.rolls,2.0,2.823333,3,3,8,41,33,0.0,Thursday,1,False,1,afternoon,0,0,0,34.06,1.0,8.98


**DINNER**

### TABLE 7

In [146]:
valentine[valentine.index == 447650659].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447650659,2019-02-15 00:45:00,2019-02-15 01:41:35,2019-02-15 00:45:00,jacks.mule,3.0,0.943056,1,2,12,13,0,0.0,Friday,2,True,1,night,0,0,0,30.78,1.0,8.97
447650659,2019-02-15 00:45:00,2019-02-15 01:41:35,2019-02-15 00:52:00,faj.for.,2.0,0.943056,2,2,12,13,1,0.0,Friday,2,True,1,night,0,0,0,30.78,1.0,0.00
447650659,2019-02-15 00:45:00,2019-02-15 01:41:35,2019-02-15 00:52:00,faj.chicken,2.0,0.943056,2,2,12,13,1,0.0,Friday,2,True,1,night,0,0,0,30.78,1.0,21.81


**CASUAL_DRINK**

### TABLE 8

In [147]:
valentine[valentine.index == 447372783].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
447372783,2019-02-14 13:03:00,2019-02-14 13:17:50,2019-02-14 13:03:00,asiago.bowtie,2.0,0.247222,1,1,3,3,0,0.0,Thursday,1,False,1,lunch,0,0,0,19.99,1.0,19.99


**LUNCH**

--------------

# Silvester

In [105]:
silvester_1gc = get_tables_gc_1(silvester_tables)
print("We have {} tables of a total of 1 guests".format(len(silvester_1gc)))

We have 33703 tables of a total of 1 guests


In [110]:
#picked_silvester_tables = random.sample(list(silvester_1gc.index), COUNT)

In [109]:
picked_silvester_tables = [363436344,
 361891515,
 361024418,
 360026487,
 359644172,
 360384742,
 360472971,
 361700721,
 362619447,
 363618332,
 359433225,
 362233587,
 361562102,
 362692290,
 362879170,
 363631395,
 363620521]

In [152]:
#TEST
picked_silvester_test_tables = random.sample(list(silvester_1gc.index), COUNT)
#picked_silvester_test_tables

In [151]:
test_s = [363870533,
 361923715,
 361814929,
 363737552,
 363753228,
 363829806,
 362363734,
 363270728,
 359456372,
 363831085]

# Table 1

In [247]:
silvester[silvester.index == 363436344].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
363436344,2019-01-03 17:39:00,2019-01-04 00:01:01,2019-01-03 17:39:00,VEGGIE.BURGER,2.0,6.366944,1,95,95,0,0.0,Thursday,False,1,afternoon,0,0,0,8.81,1.0,8.81


**Jan 03rd (Thursday), 17:39 - 00:01, total of 6 hours (mistake), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - veggie burger


**total meals**: 1 dinner meal.

**`dinner`** 

`dinner_time`, `1_meal`

**Explanation**: classic dinner for 1. Only mistake I see is they didnt close the table, and so the dwell time says 6 hours.


# Table 2

In [14]:
silvester[silvester.index == 361891515].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,v.sassafraz,4.0,0.975556,1,1,14,14,0,0.000,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,4.95
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,guinness.pt,1.0,0.975556,1,1,14,14,0,0.473,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,9.33
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:47:00,drunken.platter,2.0,0.975556,1,1,14,14,0,0.000,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,29.97
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:48:00,sirl.memphis,2.0,0.975556,1,1,14,14,0,0.000,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,27.97
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:48:00,jacks.veg,2.0,0.975556,1,1,14,14,0,0.000,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,0.00
361891515,2018-12-30 22:47:00,2018-12-30 23:45:32,2018-12-30 22:48:00,loaded.mash,2.0,0.975556,1,1,14,14,0,0.000,Sunday,6,True,1,hang_out,0,0,0,73.21,1.0,0.99


**NOT_1**

# Table 3

In [250]:
silvester[silvester.index == 361024418].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
361024418,2018-12-28 22:17:00,2018-12-28 23:13:13,2018-12-28 22:17:00,STRAW.PALOMA,3.0,0.936944,1,13,13,0,0.0,Friday,True,1,hang_out,0,0,0,29.98,1.0,13.99
361024418,2018-12-28 22:17:00,2018-12-28 23:13:13,2018-12-28 22:18:00,CHK.GOATCHS.WRAP,2.0,0.936944,1,13,13,0,0.0,Friday,True,1,hang_out,0,0,0,29.98,1.0,15.99


**Dec 28th (Friday), 22:17 - 23:13, 0.9 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - cocktail, strawberry paloma
    - meal
        - chicken wrap


**Total drinks**: 1 cocktail, **total meals**: 1 meal

**`Late dinner`** 

`late night`, `1 drink`, `1 meal`, 

**Explanation**: Another classic dinner for 1, but late at night.


# Table 4

In [251]:
silvester[silvester.index == 360026487].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
360026487,2018-12-26 17:42:00,2018-12-26 18:15:35,2018-12-26 17:42:00,FAJ.TWO.CHK.SHRMP,2.0,0.559722,1,8,8,0,0.0,Wednesday,False,1,afternoon,0,0,0,36.97,1.0,36.97
360026487,2018-12-26 17:42:00,2018-12-26 18:15:35,2018-12-26 17:42:00,FAJ.FOR.TWO,2.0,0.559722,1,8,8,0,0.0,Wednesday,False,1,afternoon,0,0,0,36.97,1.0,0.00


**Dec 26th (Wednesday), 17:42 - 18:15, 0.55 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - fajitas chicken shrimp (big meal)


**total meals**: 1 meal

**`dinner`** 

`dinner time`, `1 meal`,`big meal` 

**Explanation**: classic dinner for 1, a lot of food.


# Table 5

In [252]:
silvester[silvester.index == 359644172].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:18:00,KEITH'S TAP,1.0,4.015278,1,20,58,0,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:18:00,POTATO SKINS,2.0,4.015278,1,20,58,0,0.000,Tuesday,False,1,dinner,0,0,0,55.1,1.0,8.2
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 18:43:00,KEITH'S TAP,1.0,4.015278,2,20,58,6,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:02:00,KEITH'S TAP,1.0,4.015278,3,20,58,10,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:03:00,PEROGIES,2.0,4.015278,3,20,58,10,0.000,Tuesday,False,1,dinner,0,0,0,55.1,1.0,7.9
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 19:59:00,KEITH'S TAP,1.0,4.015278,4,20,58,24,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 20:23:00,KEITH'S TAP,1.0,4.015278,5,20,58,30,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5
359644172,2018-12-25 18:18:00,2018-12-25 22:18:55,2018-12-25 20:57:00,KEITH'S TAP,1.0,4.015278,6,20,58,38,0.591,Tuesday,False,1,dinner,0,0,0,55.1,1.0,6.5


**Dec 25th (Tuesday), 18:18 - 22:18, 4 hrs, 6 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - potatos (chips?)
    - drink
        - keiths tap, half liter
2. 25 minutes later
    - drink
        - keiths tap, half liter
3. 20 minutes later
    - meal
        - perogies, snack
4. 50 min later
    - drink
        - keiths tap, half liter
5. 25 minutes later
    - drink
        - keiths tap, half liter
6. 30 minutes later
    - drink
        - keiths tap, half liter


**total meals**: 2 snack type foods, **total drinks**: 5 beers, half liters

**`Drinking`** - Dinner time 

`dinner time`, `large amount of alcohol`, `dwell time`, `number of steps` 

**Explanation**: Christams day drinking:(, right at dinner time, eats two somewhat smaller type foods, I wouldnt say they were "meals", more drinking food. The person drinks 5 half liter beers, the first two 25 min apart, and the last three also about 25 min apart.


# Table 6

In [111]:
silvester[silvester.index == 360384742].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 17:47:00,polar.ice..,3.0,1.28,1,3,5,18,0,0.00,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,0.00
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 17:47:00,bloody.caesar,3.0,1.28,1,3,5,18,0,0.00,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,8.56
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 17:47:00,coors.light btl,1.0,1.28,1,3,5,18,0,0.34,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,5.96
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 18:04:00,italian.burger,2.0,1.28,2,3,5,18,4,0.00,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,17.62
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 18:04:00,house,2.0,1.28,2,3,5,18,4,0.00,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,0.00
360384742,2018-12-27 17:47:00,2018-12-27 19:03:48,2018-12-27 18:43:00,water.,4.0,1.28,3,3,5,18,13,0.00,Thursday,3,False,1,afternoon,0,0,0,32.14,1.0,0.00


**->DINNER**

# Table 7

In [113]:
silvester[silvester.index == 360472971].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 20:17:00,virgin.mary,4.0,1.4475,1,2,5,21,0,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,4.95
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 20:17:00,chickenribs,2.0,1.4475,1,2,5,21,0,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,29.75
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 20:17:00,side.trufle.frit,2.0,1.4475,1,2,5,21,0,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,7.65
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 20:17:00,juice,4.0,1.4475,1,2,5,21,0,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,3.00
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 20:17:00,apple.juice,4.0,1.4475,1,2,5,21,0,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,0.00
360472971,2018-12-27 20:17:00,2018-12-27 21:43:51,2018-12-27 21:21:00,detour.mg,3.0,1.4475,2,2,5,21,16,0.0,Thursday,5,False,1,dinner,0,0,0,53.35,1.0,8.00


**->DINNER**

# Table 8

In [117]:
silvester[silvester.index == 361700721].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
361700721,2018-12-30 15:14:00,2018-12-30 15:25:59,2018-12-30 15:14:00,asiago.bowtie,2.0,0.199722,1,1,2,2,0,0.0,Sunday,1,True,1,afternoon,0,0,0,24.96,1.0,24.96


**->UNK**

# Table 9

In [119]:
silvester[silvester.index == 362619447].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
362619447,2019-01-01 15:28:00,2019-01-01 16:19:06,2019-01-01 15:28:00,dime caesar,3.0,0.851667,1,3,7,11,0,0.0,Tuesday,2,False,1,afternoon,0,0,0,30.7,1.0,8.25
362619447,2019-01-01 15:28:00,2019-01-01 16:19:06,2019-01-01 15:31:00,dime caesar,3.0,0.851667,1,3,7,11,0,0.0,Tuesday,2,False,1,afternoon,0,0,0,30.7,1.0,8.25
362619447,2019-01-01 15:28:00,2019-01-01 16:19:06,2019-01-01 15:36:00,avocado toast,2.0,0.851667,2,3,7,11,1,0.0,Tuesday,2,False,1,afternoon,0,0,0,30.7,1.0,5.95
362619447,2019-01-01 15:28:00,2019-01-01 16:19:06,2019-01-01 15:50:00,dime caesar,3.0,0.851667,3,3,7,11,4,0.0,Tuesday,2,False,1,afternoon,0,0,0,30.7,1.0,8.25


**->DRINKING**

# Table 10

In [120]:
silvester[silvester.index == 363618332].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363618332,2019-01-04 11:36:00,2019-01-04 12:56:02,2019-01-04 11:41:00,trad.guac,2.0,1.333889,1,2,16,19,1,0.0,Friday,1,True,1,lunch,1,0,0,23.12,0.25,2.25
363618332,2019-01-04 11:36:00,2019-01-04 12:56:02,2019-01-04 11:51:00,chk.parmigiana,2.0,1.333889,2,2,16,19,3,0.0,Friday,1,True,1,lunch,0,0,0,23.12,1.00,20.87


**->LUNCH**

# Table 11

In [121]:
silvester[silvester.index == 359433225].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
359433225,2018-12-24 14:43:00,2018-12-24 15:15:09,2018-12-24 14:43:00,dbl.chsburg,2.0,0.535833,1,1,8,8,0,0.0,Monday,1,False,1,afternoon,0,0,0,16.68,1.0,16.68


**->UNK**

# Table 12

In [124]:
silvester[silvester.index == 362233587].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
362233587,2018-12-31 18:43:00,2018-12-31 20:36:34,2018-12-31 18:43:00,hot.tea,4.0,1.892778,1,3,10,28,0,0.0,Monday,2,False,1,dinner,0,0,0,30.7,1.0,3.99
362233587,2018-12-31 18:43:00,2018-12-31 20:36:34,2018-12-31 18:51:00,salmon.rice/veg,2.0,1.892778,2,3,10,28,2,0.0,Monday,2,False,1,dinner,0,0,0,30.7,1.0,0.00
362233587,2018-12-31 18:43:00,2018-12-31 20:36:34,2018-12-31 18:51:00,smokin.salmon,2.0,1.892778,2,3,10,28,2,0.0,Monday,2,False,1,dinner,0,0,0,30.7,1.0,23.96
362233587,2018-12-31 18:43:00,2018-12-31 20:36:34,2018-12-31 19:56:00,ws.brownie,2.0,1.892778,3,3,10,28,18,0.0,Monday,2,False,1,dinner,0,0,0,30.7,1.0,2.75


**-<DINNER**

# Table 13

In [126]:
silvester[silvester.index == 361562102].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
361562102,2018-12-30 00:26:00,2018-12-30 01:17:34,2018-12-30 00:26:13,my thai,3.0,0.859444,1,2,9,12,0,0.0,Sunday,1,True,1,night,0,0,0,13.25,1.0,6.75
361562102,2018-12-30 00:26:00,2018-12-30 01:17:34,2018-12-30 00:39:48,vodka,3.0,0.859444,2,2,9,12,3,0.0,Sunday,1,True,1,night,0,0,0,13.25,1.0,6.50


**->DRINKING**

# Table 14

In [128]:
silvester[silvester.index == 362692290].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
362692290,2019-01-01 19:46:00,2019-01-01 21:35:58,2019-01-01 19:46:00,water.,4.0,1.832778,1,2,8,26,0,0.0,Tuesday,2,False,1,dinner,0,0,0,27.18,1.0,0.00
362692290,2019-01-01 19:46:00,2019-01-01 21:35:58,2019-01-01 19:46:00,holy.smoke.burgr,2.0,1.832778,1,2,8,26,0,0.0,Tuesday,2,False,1,dinner,0,0,0,27.18,1.0,17.97
362692290,2019-01-01 19:46:00,2019-01-01 21:35:58,2019-01-01 21:01:00,extreme.cake,2.0,1.832778,2,2,8,26,18,0.0,Tuesday,2,False,1,dinner,1,0,0,27.18,1.0,9.21


**->DINNER**

# Table 15

In [132]:
silvester[silvester.index == 362879170].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
362879170,2019-01-02 12:22:00,2019-01-02 13:32:03,2019-01-02 12:22:00,bruschetta.,2.0,1.1675,1,2,14,16,0,0.0,Wednesday,2,False,1,lunch,0,0,0,36.02,0.5,6.66
362879170,2019-01-02 12:22:00,2019-01-02 13:32:03,2019-01-02 12:22:00,w.j.t.chard.lg,6.0,1.1675,1,2,14,16,0,0.0,Wednesday,2,False,1,lunch,0,0,0,36.02,1.0,8.00
362879170,2019-01-02 12:22:00,2019-01-02 13:32:03,2019-01-02 12:33:00,pad thai,2.0,1.1675,2,2,14,16,2,0.0,Wednesday,2,False,1,lunch,0,0,0,36.02,1.0,0.00
362879170,2019-01-02 12:22:00,2019-01-02 13:32:03,2019-01-02 12:33:00,pad.thai.tofu,2.0,1.1675,2,2,14,16,2,0.0,Wednesday,2,False,1,lunch,0,0,0,36.02,1.0,21.36


**->LUNCH**

# Table 16

In [133]:
silvester[silvester.index == 363631395].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363631395,2019-01-04 12:08:00,2019-01-04 13:13:50,2019-01-04 12:08:00,mexi.salad,2.0,1.097222,1,1,16,16,0,0.0,Friday,1,True,1,lunch,0,0,0,18.99,1.0,18.99


**->LUNCH**

# Table 17

In [135]:
silvester[silvester.index == 363620521].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363620521,2019-01-04 12:18:48,2019-01-04 12:58:16,2019-01-04 12:19:02,oz sapporo spc,1.0,0.657778,1,2,8,9,0,0.473,Friday,2,True,1,lunch,0,0,0,23.97,1.0,5.99
363620521,2019-01-04 12:18:48,2019-01-04 12:58:16,2019-01-04 12:20:50,morning glory,4.0,0.657778,1,2,8,9,0,0.000,Friday,2,True,1,lunch,0,0,0,23.97,1.0,4.99
363620521,2019-01-04 12:18:48,2019-01-04 12:58:16,2019-01-04 12:25:35,p.b deli sand,2.0,0.657778,2,2,8,9,1,0.000,Friday,2,True,1,lunch,0,0,0,23.97,1.0,12.99


**LUNCH**

# SILVESTER TEST ANNOTATIONS

### TABLE 1

In [154]:
silvester[silvester.index == 363870533].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 18:41:00,pink lemonade,3.0,4.658611,1,5,29,68,0,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,6.75
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 18:43:00,mac n cheese,2.0,4.658611,1,5,29,68,0,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,7.90
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 19:00:00,pink lemonade,3.0,4.658611,2,5,29,68,4,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,6.75
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 19:25:00,pink lemonade dbl,3.0,4.658611,3,5,29,68,10,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,11.50
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 20:06:00,wrhse lemonade,3.0,4.658611,4,5,29,68,20,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,12.75
363870533,2019-01-04 18:41:00,2019-01-04 23:20:31,2019-01-04 21:23:00,wrhse lemonade,3.0,4.658611,5,5,29,68,39,0.0,Friday,2,True,1,dinner,0,0,0,58.4,1.0,12.75


**DRINKING**

### TABLE 2

In [156]:
silvester[silvester.index == 361923715].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
361923715,2018-12-31 00:56:00,2018-12-31 01:48:06,2018-12-31 01:19:00,taco jerk chicken,2.0,0.868333,1,1,7,12,5,0.0,Monday,2,False,1,night,0,0,0,14.4,1.0,8.45
361923715,2018-12-31 00:56:00,2018-12-31 01:48:06,2018-12-31 01:19:00,taco jerk chicken,2.0,0.868333,1,1,7,12,5,0.0,Monday,2,False,1,night,0,0,0,14.4,1.0,5.95


**MUNCH**

### TABLE 3

In [158]:
silvester[silvester.index == 361814929].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
361814929,2018-12-30 21:05:00,2018-12-30 21:28:09,2018-12-30 21:05:00,caesar,2.0,0.385833,1,2,2,5,0,0.0,Sunday,3,True,1,hang_out,0,0,0,29.83,1.0,0.00
361814929,2018-12-30 21:05:00,2018-12-30 21:28:09,2018-12-30 21:05:00,fingers.sauced,2.0,0.385833,1,2,2,5,0,0.0,Sunday,3,True,1,hang_out,1,0,0,29.83,1.0,16.87
361814929,2018-12-30 21:05:00,2018-12-30 21:28:09,2018-12-30 21:06:00,cherry.mule,3.0,0.385833,1,2,2,5,0,0.0,Sunday,3,True,1,hang_out,0,0,0,29.83,1.0,8.97
361814929,2018-12-30 21:05:00,2018-12-30 21:28:09,2018-12-30 21:20:00,pop.,4.0,0.385833,2,2,2,5,3,0.0,Sunday,3,True,1,hang_out,0,0,0,29.83,1.0,3.99


**DINNER**

### TABLE 4

In [159]:
silvester[silvester.index == 363737552].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363737552,2019-01-04 19:21:00,2019-01-04 20:54:35,2019-01-04 19:37:00,water.,4.0,1.559722,1,3,9,22,4,0.0,Friday,2,True,1,dinner,0,0,0,25.17,1.0,0.00
363737552,2019-01-04 19:21:00,2019-01-04 20:54:35,2019-01-04 19:39:00,jacks.panchees,2.0,1.559722,1,3,9,22,4,0.0,Friday,2,True,1,dinner,0,0,0,25.17,0.5,4.50
363737552,2019-01-04 19:21:00,2019-01-04 20:54:35,2019-01-04 19:56:00,cheese.burger,2.0,1.559722,2,3,9,22,8,0.0,Friday,2,True,1,dinner,0,0,0,25.17,1.0,16.68
363737552,2019-01-04 19:21:00,2019-01-04 20:54:35,2019-01-04 20:18:00,pop.,4.0,1.559722,3,3,9,22,13,0.0,Friday,2,True,1,dinner,0,0,0,25.17,1.0,3.99


**DINNER**

### TABLE 5

In [160]:
silvester[silvester.index == 363753228].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363753228,2019-01-04 18:29:00,2019-01-04 20:20:33,2019-01-04 18:29:00,bubble.bowl,3.0,1.859167,1,4,3,26,0,0.0,Friday,2,True,1,dinner,0,0,0,29.17,0.5,8.50
363753228,2019-01-04 18:29:00,2019-01-04 20:20:33,2019-01-04 18:36:00,broc.bites,2.0,1.859167,2,4,3,26,1,0.0,Friday,2,True,1,dinner,0,0,0,29.17,0.2,2.30
363753228,2019-01-04 18:29:00,2019-01-04 20:20:33,2019-01-04 18:36:00,calamari,2.0,1.859167,2,4,3,26,1,0.0,Friday,2,True,1,dinner,1,0,0,29.17,0.2,2.44
363753228,2019-01-04 18:29:00,2019-01-04 20:20:33,2019-01-04 18:57:00,chk.goatchs.wrap,2.0,1.859167,3,4,3,26,6,0.0,Friday,2,True,1,dinner,0,0,0,29.17,1.0,13.59
363753228,2019-01-04 18:29:00,2019-01-04 20:20:33,2019-01-04 20:07:00,ws.brownie,2.0,1.859167,4,4,3,26,23,0.0,Friday,2,True,1,dinner,0,0,0,29.17,1.0,2.34


**DINNER**

### TABLE 6

In [161]:
silvester[silvester.index == 363829806].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 20:59:00,pop.,4.0,1.852222,1,4,7,26,0,0.0,Friday,3,True,1,dinner,0,0,0,59.74,1.0,3.99
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 20:59:00,water.,4.0,1.852222,1,4,7,26,0,0.0,Friday,3,True,1,dinner,0,0,0,59.74,1.0,0.00
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 21:04:00,pop.,4.0,1.852222,2,4,7,26,1,0.0,Friday,3,True,1,dinner,0,0,0,59.74,1.0,3.99
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 21:15:00,jacks.veg.nachos,2.0,1.852222,3,4,7,26,3,0.0,Friday,3,True,1,dinner,1,0,0,59.74,1.0,23.76
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 21:15:00,pout.classic,2.0,1.852222,3,4,7,26,3,0.0,Friday,3,True,1,dinner,0,0,0,59.74,1.0,9.53
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 21:15:00,chk.goatchs.wrap,2.0,1.852222,3,4,7,26,3,0.0,Friday,3,True,1,dinner,0,0,0,59.74,1.0,15.99
363829806,2019-01-04 20:59:00,2019-01-04 22:50:08,2019-01-04 22:20:00,kid.brownie,2.0,1.852222,4,4,7,26,19,0.0,Friday,3,True,1,dinner,0,1,0,59.74,1.0,2.48


**NOT_1**

### TABLE 7

In [162]:
silvester[silvester.index == 362363734].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,oz sapporo spc,1.0,1.524722,1,3,14,21,0,0.473,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,5.99
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,mon bellini,3.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,6.50
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,oz shock top,1.0,1.524722,1,3,14,21,0,0.473,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,6.75
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,mon bellini,3.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,6.50
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,pickle caesar,3.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,7.99
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,oz guinness,1.0,1.524722,1,3,14,21,0,0.591,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,8.00
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,macch caramel,4.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,4.49
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,mon bellini,3.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,6.50
362363734,2018-12-31 21:49:25,2018-12-31 23:20:54,2018-12-31 21:50:37,mon bellini,3.0,1.524722,1,3,14,21,0,0.000,Monday,11,False,1,hang_out,0,0,0,111.17,1.0,6.50


**NOT_1**

### TABLE 8

In [164]:
silvester[silvester.index == 363270728].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,total_meal_steps,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,max_items_per_step,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,,,
363270728,2019-01-03 16:25:00,2019-01-03 17:23:21,2019-01-03 16:25:08,mill st org tap,1.0,0.9725,1,1,14,14,0,0.414,Thursday,1,False,1,afternoon,0,0,0,7.25,1.0,7.25


**CASUAL_DRINK**